In [56]:
import timeit
import numpy as np
import pandas as pd
from matplotlib import font_manager, rc

# 배치 경사 하강법
def batch_gradient_descent(X : np.ndarray, y : np.ndarray, theta, alpha, m, numIterations, batchSize):
    xTrans = X.transpose()
    for i in range(0, numIterations):
        hypothesis = np.dot(X, theta)
        loss = hypothesis - y
        # avg cost per example (the 2 in 2*m doesn't really matter here.
        # But to be consistent with the gradient, I include it)
        cost = np.sum(loss ** 2) / (2 * m)
        # print("Iteration %d | Cost: %f" % (i, cost))
        # avg gradient per example
        gradient = np.dot(xTrans, loss) / m
        # update
        theta = theta - alpha * gradient
    # print("Iteration %d | Cost: %f" % (i, cost))
    return ("배치" , cost, theta)


# 확률적 경사 하강법
def stochastic_gradient_descent(X : np.ndarray, y : np.ndarray, theta, alpha, m, numIterations, batchSize):
    xTrans = X.transpose()
    for i in range(0, numIterations):
        for j in range(0, m):
            hypothesis = np.dot(X[j:j + 1, :], theta)
            loss = hypothesis - y[j:j + 1]
            # avg cost per example (the 2 in 2*m doesn't really matter here.
            # But to be consistent with the gradient, I include it)
            cost = np.sum(loss ** 2) / (2 * m)
            # avg gradient per example
            gradient = np.dot(xTrans[:, j:j + 1], loss) / m
            # update
            theta = theta - alpha * gradient
    # print("Iteration %d | Cost: %f" % (i, cost))
    return ("확률적" , cost, theta)

# 미니 배치 경사 하강법
def mini_batch_gradient_descent(X : np.ndarray, y : np.ndarray, theta, alpha, m, numIterations, batchSize):
    xTrans = X.transpose()
    for i in range(0, numIterations):
        for j in range(0, m, batchSize):
            hypothesis = np.dot(X[j:j + batchSize, :], theta)
            loss = hypothesis - y[j:j + batchSize]
            # avg cost per example (the 2 in 2*m doesn't really matter here.
            # But to be consistent with the gradient, I include it)
            cost = np.sum(loss ** 2) / (2 * m)
            # avg gradient per example
            gradient = np.dot(xTrans[:, j:j + batchSize], loss) / m
            # update
            theta = theta - alpha * gradient
    # print("Iteration %d | Cost: %f" % (i, cost))
    return ("미니 배치" , cost, theta)

# 미니 배치 경사 하강법 + 미니 배치에서 데이터 30개를 랜덤하게 뽑아서 사용
def mini_batch_random_gradient_descent(X : np.ndarray, y : np.ndarray, theta, alpha, m, numIterations, batchSize):
    xTrans = X.transpose()
    for i in range(0, numIterations):
        for j in range(0, m, batchSize):
            # 0~99까지의 중복되지 않는 랜덤한 정수 30개 생성
            random_idx_list = np.random.choice(100, 30, replace=False)

            # 배열에서 random_idx_list 안에 있는 인덱스만 추출
            hypothesis = np.dot(X[j:j + batchSize, :][random_idx_list], theta)
            loss = hypothesis - y[j:j + batchSize][random_idx_list]
            # avg cost per example (the 2 in 2*m doesn't really matter here.
            # But to be consistent with the gradient, I include it)
            cost = np.sum(loss ** 2) / (2 * m)
            # avg gradient per example
            gradient = np.dot(xTrans[:, j:j + batchSize][:, random_idx_list], loss) / m
            # update
            theta = theta - alpha * gradient
    # print("Iteration %d | Cost: %f" % (i, cost))
    return ("미니 배치 랜덤" , cost, theta)

# 경사하강법 속도 및 정확도 비교
def checkSpeedAndEfficiency(func_list, X, y, theta, alpha, m, numIterations, batchSize):
    test_list = []
    for func in func_list:
        
        start = timeit.default_timer()
        result = func(X, y, theta, alpha, m, numIterations, batchSize)
        stop = timeit.default_timer()
        
        name, cost, th = result
        running_time = stop - start
        
        cost_per_running_time = cost * running_time
        print(name, f"경사 하강법 (에포크 {numIterations})")
        print("시간 (running_time)                    :", format(running_time, ".15f"))
        print("비용 (cost)                            :", format(cost, ".15f"))
        print("효율 (cost * running_time)             :", format(cost_per_running_time, ".15f"))
        print()
        # test_list.append({"name": name, "time": running_time, "cost": cost, "cost_per_time": cost_per_running_time})
        test_list.append({"name": name, "cost_per_time": cost_per_running_time, running_time: running_time, cost: cost})
        test_list = sorted(test_list, key=lambda x: x["cost_per_time"])
    
    print("-"*60)
    print("효율 순서")
    for item in test_list:
        # print(f"{count}." , item["name"], ":", format(item["cost_per_time"], ".15f"))
        print( format(item["cost_per_time"], ".15f"), "(", item["name"], ")")


# ----------------------------------------------------------------------------------------------------------------------

# func_list = [batch_gradient_descent]

# # 함수 리스트
func_list = [batch_gradient_descent, \
            stochastic_gradient_descent, \
            mini_batch_gradient_descent, \
            mini_batch_random_gradient_descent]

# 데이터 로드
df = pd.read_csv('data/byong_data_set1.csv')

# 데이터 분리
X = df[['height', 'weight']].values
y = df[['bmi']].values
m, n = np.shape(X) # m : 데이터 개수, n : 특성 개수
numIterations = 5 # 에포크
theta = np.ones(n).reshape(-1, 1) # 가중치
alpha = 0.00001 # 학습률
batchSize = 100 # 미니 배치 크기

# 체크 
checkSpeedAndEfficiency(func_list, X, y, theta, alpha, m, numIterations, 100)




배치 경사 하강법 (에포크 6000)
시간 (running_time)                    : 16.021210699982475
비용 (cost)                            : 1.271536471392198
효율 (cost * running_time)             : 20.371553720886645



KeyboardInterrupt: 